# Augmenter l'intéractivité de vos notebooks Jupyter

Nous vous présentons ici comment les widgets peuvent vous permettre d'enrichir vos notebooks. 

Ce notebook complète l'[article publié](https://makina-corpus.com/blog/metier/2019/augmenter-linteractivite-de-vos-notebooks-jupyter-1) sur le blog de Makina Corpus. 

## Dépendances

Si vous n'utilisez pas les environnements pré-configurés mis à votre disposition dans le répertoire GitHub, vous aurez besoin de vérifier l'installation de certaines bibliothèques Python.

Voici la liste des dépendances de ce notebook :
   - `ipywidgets`
   - `numpy`
   - `matplotlib`
   - `bqplot`


In [1]:
import ipywidgets as widgets
from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt
import bqplot as bq
import random 

## Découvrir la fonction `interact`

`interact` est une fonction de la bibliothèque `ipywidgets` qui crée automatiquement un contrôle permettant l'affichage interactif de la sortie d'une fonction. C'est le moyen le plus simple de commencer avec les widgets Ipython. 

La méthode `interact` prend en entrée le nom de la fonction qu'elle contrôle, ainsi que les valeurs par défaut des variables d'entrée de cette fonction. En fonction de ces valeurs par défaut, le type de contrôle affiché va varier. 

| Variable d'entrée  |  Contrôle affiché |
|:---|:---|
| Un booléen | Une case à cocher  |
| Une chaîne de caractères | Une zone de texte   |
| Une valeur entière ou un tuple d'entiers : (min, max) ou (min, max, step) | Un curseur pour la sélection d'un entier  |
| Une valeur réelle ou un tuple de réels : (min, max) ou (min, max, step)  |  Un curseur pour la sélection d'un flottant |
| Une liste ou un dictionnaire | Une liste déroulante   |


### Un premier exemple simple avec une variable en entrée
#### Définition d'une fonction

In [2]:
# My function
def f(x):
    """
    This function returns directly its input parameter.
    """
    return x

#### `Interact` avec un entier

In [3]:
# The input is an integer
interact(f, x=10)

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

<function __main__.f(x)>

#### `Interact` avec un booléen

In [4]:
# The input is a boolean
interact(f, x=True)

interactive(children=(Checkbox(value=True, description='x'), Output()), _dom_classes=('widget-interact',))

<function __main__.f(x)>

#### `Interact` avec une chaîne de caractères

In [5]:
# The input is a string
interact(f, x='My text')

interactive(children=(Text(value='My text', description='x'), Output()), _dom_classes=('widget-interact',))

<function __main__.f(x)>

#### `Interact` avec un dictionnaire

In [6]:
# The input is a dictionnary
interact(f, x={"one":1, "two":2, "three":3})

interactive(children=(Dropdown(description='x', options={'one': 1, 'two': 2, 'three': 3}, value=1), Output()),…

<function __main__.f(x)>

### Avec plusieurs variables en entrée

La fonction contrôlée par `interact` peut avoir plusieurs variables d'entrée. Dans ce cas, chacune de ces variables pourra être sélectionnée par un contrôle dédié. 

Exemple :

In [7]:
# Function definition
def signal_plot(amplitude, color):
    """
    Draw a signal plot
    :param amplitude : signal amplitude
    :param color : line color
    """
    # Create a figure
    fig, ax = plt.subplots(figsize=(5, 4))
    # Add a grid
    ax.grid(color='#EEEEEE', linewidth=2, linestyle='solid')
    # Define the x range
    x = np.linspace(0, 10, 1000)
    # Plot the sinusoid
    ax.plot(x, amplitude * np.sin(x), color=color, lw=5, alpha=0.6)
    # Define the x and y limits
    ax.set_xlim(0, 10)
    ax.set_ylim(-1.1, 1.1)

    
# Interact call
interact(signal_plot,
         amplitude=(0, 1.0, 0.1),
         color=['blue', 'green', 'red'])

interactive(children=(FloatSlider(value=0.5, description='amplitude', max=1.0), Dropdown(description='color', …

<function __main__.signal_plot(amplitude, color)>

La fonction `interact` est un raccourci vers un ensemble de widgets graphiques avec des choix faits par défaut selon le type d’objet (*int, float, bool, list, etc*) passé à la fonction associée. Il est possible d’avoir plus de libertés dans ces choix en paramétrant le widget à la main comme nous allons le voir maintenant.

## Combiner les contrôles

Dans cette partie, nous allons créer une petite interface utilisateur qui permettra la visualisation de marches aléatoires. L'idée est de générer un trajet d'un nombre aléatoire de pas tiré dans un intervalle choisi par l'utilisateur, et de permettre le changement de couleur ou de style du tracé.

Voici l'aperçu de ce à quoi nous voulons arriver :

![](img/randomwalk_ui.png)

Cette interface se compose d'un générateur de nombre aléatoire, d'un cadre central pour l'affichage graphique, et d'un panneau pour la configuration des options graphiques. 

### Créer un générateur de nombre aléatoire

L'idée ici est de combiner plusieurs contrôles afin de permettre à l'utilisateur de choisir un intervalle dans un lequel un nombre sera tiré aléatoirement. Nous avons donc besoin :
- d'un sélecteur permettant de choisir un intervalles de nombres entiers ;
- d'un bouton ;
- et d'une zone de texte pour afficher le résultat.

L'ensemble des contrôles disponibles dans la bibliothèque `ipywidgets` sont listés et documentés [**ici**](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html).

#### Initialisation d'un sélecteur d'entiers

Pour le moment, rien de compliqué. Nous initialisons seulement le contrôle dont nous avons besoin, à savoir un `IntRangeSlider`.

In [8]:
# Create a slider to select a range
range_layout = widgets.Layout(
    display='flex',
    flex_flow='column',
    height='230px',
    width='130px'
)

my_range = widgets.IntRangeSlider(
    description='Intervalle choisi :',
    min=0,
    max=10000,
    value=(1000,5000),
    style={'description_width': 'initial'},
    orientation='vertical',
    layout=range_layout
)

# display(my_range) # to show the widget

L'intervalle choisi est accessible sous forme d'un tuple par l'attribut `value`.

In [9]:
my_range.value

(1000, 5000)

#### Création d'un bouton

Nous faison de même pour initialiser le bouton de notre interface.

In [10]:
# Create a button
button_layout = widgets.Layout(
    width='130px'
)

my_button = widgets.Button(
    description='Générer',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Générer un nombre aléatoire',
    layout=button_layout
)

# display(my_button) # to show the widget

#### Création d'une zone de texte 

Nous ajoutons maintenant une zone de texte pour afficher le résultat.

In [11]:
my_text = widgets.IntText(
    description = 'Résultat :',
    disabled = True,
    style={'description_width': 'initial'},
    layout=button_layout
)

# display(my_text) # to show the widget

Le contenu de cette zone de texte est accessible par l'attribut `value`.

In [12]:
my_text.value

0

#### Communication des contrôles entre eux

Nous voulons désormais que nos trois contrôles communiquent entre eux : au clic sur le bouton, un nombre doit être tiré dans l'intervalle du sélecteur et affiché dans le champ de résultat. 

Pour cela, les boutons de la bibliothèque `ipywidgets` possèdent une méthode `on_click` permettant de gérer les événements qui doivent avoir lieu au clic. Cette méthode prend en paramètre le nom de la fonction à exécuter.

In [13]:
def on_button_clicked(event):
    """
    Function called by click on the button
    """
    # Get the selected range 
    my_min = my_range.value[0]
    my_max = my_range.value[1]
    # If a correct range is selected
    if(my_min < my_max):
        # Get a random int in this range
        my_nb = np.random.randint(my_min, my_max)
        # Display this number in the text area
        my_text.value = my_nb
        # Update the button style
        my_button.button_style = 'success'
        my_button.icon = 'check'
    else:
        # Update the button style
        my_button.button_style = 'danger'
        my_button.icon = ''
  
# Define the 'on_click' event    
my_button.on_click(on_button_clicked)

# Display the three widgets
display(my_range)
display(my_button)
display(my_text)

IntRangeSlider(value=(1000, 5000), description='Intervalle choisi :', layout=Layout(display='flex', flex_flow=…

Button(description='Générer', layout=Layout(width='130px'), style=ButtonStyle(), tooltip='Générer un nombre al…

IntText(value=0, description='Résultat :', disabled=True, layout=Layout(width='130px'), style=DescriptionStyle…

#### Encapsulation des contrôles 

Ces trois contrôles peuvent désormais être assemblés dans une boîte commune (`Box`, `VBox` ou `HBox`) qui servira à construire l'interface finale.

In [14]:
box_layout = widgets.Layout(
            width='135px'
)

my_LVBox = widgets.VBox(
    [my_range, my_button, my_text],
    layout=box_layout
)

# display(my_LVBox) # to show the widget

### Afficher une marche aléatoire à partir du nombre généré

Le résultat que nous obtenons grâce à notre générateur de nombres aléatoires va désormais nous servir au tracé d'une marche aléatoire. 

Tout d'abord, voici la fonction de marche aléatoire que nous utiliserons :

In [15]:
def get_random_walk(n):  
    """
    This function creates two array containing x and y coordinates of the random walk. 
    :param n : number of steps
    """
    #creating two arrays for containing x and y coordinates 
    #of size equals to the number of size and filled up with 0's 
    x = np.zeros(n) 
    y = np.zeros(n) 

    # filling the coordinates with random variables 
    for i in range(1, n): 
        val = random.randint(1, 4) 
        if val == 1: 
            x[i] = x[i - 1] + 1
            y[i] = y[i - 1] 
        elif val == 2: 
            x[i] = x[i - 1] - 1
            y[i] = y[i - 1] 
        elif val == 3: 
            x[i] = x[i - 1] 
            y[i] = y[i - 1] + 1
        else: 
            x[i] = x[i - 1] 
            y[i] = y[i - 1] - 1
            
    return x,y

Cette fonction prend en entrée un nombre de pas et retourne deux tableaux contenant les coordonnées x et y du tracé.

#### `bqplot`, un "graphique widget"

Pour afficher le tracé, nous utilisons la bibliothèque `bqplot`. Dans notre exemple, elle va nous permettre d'intéragir avec le graphique (changer le tracé, modifier la couleur, etc) sans avoir à recharger à chaque fois toute la figure (ce que `Matplotlib` nous obligerait à faire). 

In [16]:
# Initialize the random walk with 0 steps
walk_x, walk_y = get_random_walk(0)

# Use linear scales
sc_x = bq.LinearScale()
sc_y = bq.LinearScale()

# Create the line with the coordinates of the random walk
walk = bq.Lines(x=walk_x, y=walk_y, scales={'x': sc_x,'y': sc_y}, opacities=[0.6])

# Define axis
ax_x = bq.Axis(scale=sc_x)
ax_y = bq.Axis(scale=sc_y, orientation='vertical')

# Create a figure
fig = bq.Figure(marks=[walk], axes=[ax_x, ax_y],
                fig_margin=dict(top=20, bottom=20, left=20, right=20))

# Fix the figure size
fig.layout.height = '450px'
fig.layout.width = '450px'

fig

Figure(axes=[Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale())], fig_margin={'top':…

#### Lier le générateur de nombre aléatoire à l'affichage graphique

Maintenant, grâce à `bqplot`, nous pouvons facilement redessiner le tracé à chaque fois qu'un nouveau nombre alétoire est généré. 

Pour cela, nous utilisons la méthode `observe` de notre zone de texte qui permet d'appeler une fonction à chaque fois que sa valeur change. 

In [17]:
def on_value_change(change):
    """
    Update the random walk when a new number is generated.
    """
    # Random number
    n = my_text.value
    # Calculate a new random walk
    wx, wy = get_random_walk(n)
    # Update the plot
    walk.x = wx
    walk.y = wy
    
my_text.observe(on_value_change,'value')

### Rendre configurable des options graphiques

Dernière partie de notre interface, nous allons ajouter quelques contrôles pour pouvoir changer facilement la couleur ou/et le style du tracé.

#### Changer la couleur du tracé

In [18]:
layout =  widgets.Layout(
            display='flex',
            flex_flow='column',
            height='60px',
            width='100px'
        )

my_color = widgets.Dropdown(
    options=['blue', 'green', 'red'],
    value='blue',
    description='Couleur:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=layout
)

def on_color_change(change):
    """
    Change the plot color
    """
    walk.colors = [my_color.value]
        
my_color.observe(on_color_change, 'value')

#### Changer le style du tracé

In [19]:
my_line = widgets.Dropdown(
    options=['solid', 'dashed', 'dotted', 'dash_dotted'],
    value='solid',
    description='Style des lignes:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=layout
)

def on_line_change(change):
    """
    Change the line style
    """
    walk.line_style = my_line.value
        
my_line.observe(on_line_change, 'value')

#### Ajouter une image et encapsuler les contrôles

In [20]:
# Open the file containing our image and read it
with open('img/pedestrians-1209316_1920.jpg','rb') as my_file:
    img = my_file.read()

# Create an Image widget to display it in the UI
my_img = widgets.Image(
    value=img,
    format='jpg'
)

# Create a box containing the three widgets
b_layout = widgets.Layout(
    width='200px'
)

my_RVBox = widgets.VBox(
    children=[my_img, my_color, my_line],
    layout=b_layout
)

### Interface complète

Ca y est, nous avons tous les morceaux de notre interface. Pour la construire, il nous suffit maintenant de les rassembler dans un `AppLayout`.

In [21]:
widgets.AppLayout(
    header=None,
    left_sidebar=my_LVBox,
    center=fig,
    right_sidebar=my_RVBox,
    footer=None,
    align_items="center",
    width='85%'
)

AppLayout(children=(VBox(children=(IntRangeSlider(value=(1000, 5000), description='Intervalle choisi :', layou…